# Chemical Structure Inputs for PUG-REST

adapted from https://chem.libretexts.org/Courses/Intercollegiate_Courses/Cheminformatics

## Objectives

- Use SMILES and InChI strings to specify the input compound for a PUG-REST request.
- Use a structure-data (SD) file to specify the input compound for a PUG-REST request.
- Learn to submit a PUG-REST request using the HTTP-POST method.
<br>
<br>

You can use a chemical structure as an input for a PUG-REST request.  PUG-REST accepts some popular chemical structure line notations such as SMILES and InChI strings.  It is also possible to use an Structure-Data File (SDF) as a structure input.<br><br>
To learn how to specify the structure input in a PUG-REST request, one needs to know that there are two methods by which data are transferred from clients (users) and servers (PubChem) through PUG-REST.  Discussing what these methods are in detail is beyond the scope of this material, and it is enough to know three things:<br>
- When you make a PUG-REST request by typing the request URL in the address bar of your web browser (such as Google Chrome, MS Internet Explorer), the HTTP GET method is used<br>
- The HTTP GET method transfers information encoded in a single-line URL.<br>
- Some chemical structure inputs are not appropriate to encode in a single-line URL (because they may contain special characters not compatible with the URL syntax, span over multiple lines, or too long), and the HTTP POST needs to be used for such cases.

For more information on HTTP GET and POST, read the following documents.

- HTTP request methods (https://www.w3schools.com/tags/ref_httpmethods.asp)
- Get vs. POST (https://www.diffen.com/difference/GET-vs-POST-HTTP-Requests)


Here, import the Requests library, necessary to make web service requests to PubChem.

In [16]:
import requests

## 1. Using the HTTP GET method.

## 1-(1) Structure encoded in the URL path.

In some cases, you can encode a chemical structure in the PUG-REST request URL path as in the following example.

In [17]:
prolog = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug'

In [18]:
smiles1 = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"
url = prolog + "/compound/smiles/" + smiles1 + "/cids/txt"
print(url)

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CC(C)CC1=CC=C(C=C1)C(C)C(=O)O/cids/txt


This request URL returns ibuprofen (CID 3672).

In [21]:
res = requests.get(url)
print(res.text)

3672



Try to run the following

In [24]:
smiles2 = "CC1=C([C@@](SC1=O)(C)/C=C(\C)/C=C)O"

url = prolog + "/compound/smiles/" + smiles2 + "/cids/txt"
res = requests.get(url)
print(res.text)

Status: 400
Code: PUGREST.BadRequest
Message: Unable to standardize the given structure - perhaps some special characters need to be escaped or data packed in a MIME form?
Detail: error: 
Detail: status: 400
Detail: output: Caught ncbi::CException: Standardization failed
Detail: Output Log:
Detail: Record 1: Warning: Cactvs Ensemble cannot be created from input string
Detail: Record 1: Error: Unable to convert input into a compound object
Detail: 
Detail: 



<br>Note in the above example that the SMILES string contains special characters. In this case a forward slash ("/"), which is also used in the URL path.  These special characters conflict with the PUG-REST request URL syntax, causing an error when used in the PUG-REST request URL.

## 1-(2) Structure encoded as a URL argument

To circumvent the issue mentioned above, the SMILES string may be encoded as the URL argments (as an optional parameter followed by the "?" character).

In [25]:
url = prolog + "/compound/smiles/cids/txt?" + "smiles=" + smiles2
print(url)
res2 = requests.get(url)
print(res2.text)

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/cids/txt?smiles=CC1=C([C@@](SC1=O)(C)/C=C(\C)/C=C)O
135403829



## 1-(3) Structure passed in a dictionary

It is also possible to pass the structure query as a key-value pair in a dictionary.  The following example does the same task as the previous example does.

In [26]:
url = prolog + "/compound/smiles/cids/txt"
struct = { 'smiles': smiles2 }
res3 = requests.get(url, params = struct)
print(res3.text)

135403829



The object returned from a web service request (res, res2, and res3 in our examples) contains information on the request URL through which the data have been retrieved.  This information can be accessed using the ".url" attribute of the object, as shown in this example:

In [27]:
print(smiles2)    # the orginal smiles string unencoded
print(res2.url)   # from (request 2) structure encoded as a URL argument
print()
print(struct)     # to show the smiles string in the dictionary is unencoded for URL
print(res3.url)   # from (request 3) structure passed in a dictionary

CC1=C([C@@](SC1=O)(C)/C=C(\C)/C=C)O
https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/cids/txt?smiles=CC1=C(%5BC@@%5D(SC1=O)(C)/C=C(%5CC)/C=C)O

{'smiles': 'CC1=C([C@@](SC1=O)(C)/C=C(\\C)/C=C)O'}
https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/cids/txt?smiles=CC1%3DC%28%5BC%40%40%5D%28SC1%3DO%29%28C%29%2FC%3DC%28%5CC%29%2FC%3DC%29O


From these two URLs, we can see two important things:
- When the structure is passed using a key-value pair in a dictionary (i.e., "res3"), the structure is automatically encoded as a URL argument (after the "?" mark).
- When the structure is passed in a dictionary, the special characters in the SMILES string are coverted according to the URL encoding rules: https://www.w3schools.com/tags/ref_urlencode.asp.  [for example, the equal sign "=" changes into "%3D", and "(" into "%28", "/" into "%2F", etc]
<br>


It illustrates that the last two approaches using HTTP GET are essentially the same.

__Exercise 1a__ Retrieve (in the CSV (comma-separated values) format) the Hydrogen bond donor and acceptor counts, TPSA, and XLogP of the chemical represented by the SMILES string: "C1=CC(=C(C=C1Cl)O)OC2=C(C=C(C=C2)Cl)Cl".  When you construct a PUG-REST url for this request, encode the structure in the URL path. 

In [60]:
# Write your code in this cell.
smiles3 = "C1=CC(=C(C=C1Cl)O)OC2=C(C=C(C=C2)Cl)Cl"

pugrest = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
pugin   = "compound/smiles"
pugoper = "property/HBondDonorCount,HBondDonorCount,XLogP,TPSA"
pugout  = "csv?smiles="

# url = "/".join( [pugrest, pugin, pugoper, pugout, smiles3] ) 
# print(url)

url = "/".join( [pugrest, pugin, pugoper, pugout] )   # Construct the URL
print(url,'\n')

url += smiles3
print(url,'\n')

print("-" * 30,'\n')   # Print "-" 30 times (to print a line for readability)

res = requests.get(url)
print(res.text)

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/property/HBondDonorCount,HBondDonorCount,XLogP,TPSA/csv?smiles= 

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/property/HBondDonorCount,HBondDonorCount,XLogP,TPSA/csv?smiles=C1=CC(=C(C=C1Cl)O)OC2=C(C=C(C=C2)Cl)Cl 

------------------------------ 

"CID","HBondDonorCount","HBondDonorCount","XLogP","TPSA"
5564,1,1,5.000,29.5



**Exercise 1b** Get the CID corresponding to the following InChI string, using the HTTP GET method.  Pay attention to the case sensitivity of the URL parameter part after the "?" mark.

In [ ]:
inchi = "InChI=1S/C17H14O4S/c1-22(19,20)14-9-7-12(8-10-14)15-11-21-17(18)16(15)13-5-3-2-4-6-13/h2-10H,11H2,1H3"

In [ ]:
# Write your code in this cell




## 2. Using the HTTP POST method

## 2-(1) Comparison of HTTP POST and GET

All the three examples above use the HTTP GET method, as implied in the use of "requests.get()".  Alternatively, one can use the HTTP POST method.  For example, the following example returns the identical result as the last two HTTP GET examples.

In [30]:
url = prolog + "/compound/smiles/cids/txt"
struct = { 'smiles': smiles2 }
res = requests.post(url, params = struct)   # the SMILES as a URL parameter
print(res.url)
print(res.text)

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/cids/txt?smiles=CC1%3DC%28%5BC%40%40%5D%28SC1%3DO%29%28C%29%2FC%3DC%28%5CC%29%2FC%3DC%29O
135403829



In [31]:
url = prolog + "/compound/smiles/cids/txt"
struct = { 'smiles': smiles2 }
res = requests.post(url, data = struct)    # the SMILES as data
print(res.url)
print(res.text)

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/cids/txt
135403829



Note that the first one passes the input SMILES string as a parameter, while the second one passes it as data.  Because of this, the URL stored in the "res.url" variable in the second code does not contain structure informaion.

## 2-(2) HTTP POST for multi-line structure input

The HTTP POST method should be used if the input molecular structure for PUG-REST request span over multiple lines (e.g., stored in a structure-data file (SDF) format).  The SDF file contains structure information of a molecule in a multi-line format, along with other data.

In [32]:
mysdf = '''1983
  -OEChem-07241917072D

 20 20  0     0  0  0  0  0  0999 V2000
    2.8660   -2.5950    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    4.5981    1.4050    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    2.8660    1.4050    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    2.8660    0.4050    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    3.7320   -0.0950    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0000   -0.0950    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    3.7320   -1.0950    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.0000   -1.0950    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.8660   -1.5950    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    3.7320    1.9050    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    3.7320    2.9050    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    4.2690    0.2150    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.4631    0.2150    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    2.3291    1.7150    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    4.2690   -1.4050    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.4631   -1.4050    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    4.3520    2.9050    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    3.7320    3.5250    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    3.1120    2.9050    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    2.3291   -2.9050    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  9  1  0  0  0  0
  1 20  1  0  0  0  0
  2 10  2  0  0  0  0
  3  4  1  0  0  0  0
  3 10  1  0  0  0  0
  3 14  1  0  0  0  0
  4  5  2  0  0  0  0
  4  6  1  0  0  0  0
  5  7  1  0  0  0  0
  5 12  1  0  0  0  0
  6  8  2  0  0  0  0
  6 13  1  0  0  0  0
  7  9  2  0  0  0  0
  7 15  1  0  0  0  0
  8  9  1  0  0  0  0
  8 16  1  0  0  0  0
 10 11  1  0  0  0  0
 11 17  1  0  0  0  0
 11 18  1  0  0  0  0
 11 19  1  0  0  0  0
M  END
> <PUBCHEM_COMPOUND_CID>
1983

> <PUBCHEM_COMPOUND_CANONICALIZED>
1

> <PUBCHEM_CACTVS_COMPLEXITY>
139

> <PUBCHEM_CACTVS_HBOND_ACCEPTOR>
2

> <PUBCHEM_CACTVS_HBOND_DONOR>
2

> <PUBCHEM_CACTVS_ROTATABLE_BOND>
1
$$$$
'''

In this example, the triple quotes (''') are used to enclose a multi-line string.  This multi-line sdf data is used as an input for a PUG-REST request through the HTTP POST.

In [33]:
url = prolog + "/compound/sdf/cids/txt"
mydata = { 'sdf': mysdf }
res = requests.post(url, data=mydata) # the multiline sdf as URL data
print(res.url)
print(res.text)

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/sdf/cids/txt
1983



## 2-(3) HTTP POST for SDF file input

One may want to use the structure stored in a file as the input for a PUG-REST request.  The following code shows how to read an SDF file into a variable.  This code cell assumes that the 'Structure2D_CID_5288826.sdf' file is in the current directory.  The file can be downloaded from the **Chapter 2 Assignments** page.

In [34]:
with open('Structure2D_CID_5288826.sdf', 'r') as file:
    mysdf = file.read()
    
print(mysdf)

5288826
  -OEChem-08171913162D

 40 44  0     1  0  0  0  0  0999 V2000
    2.2314    0.0528    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    2.0000   -2.4021    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    2.0000    2.4021    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    6.1607   -0.9511    0.0000 N   0  0  3  0  0  0  0  0  0  0  0  0
    3.6897   -0.4755    0.0000 C   0  0  1  0  0  0  0  0  0  0  0  0
    4.5133   -0.9511    0.0000 C   0  0  2  0  0  0  0  0  0  0  0  0
    5.3370   -0.4755    0.0000 C   0  0  1  0  0  0  0  0  0  0  0  0
    2.8660   -0.9511    0.0000 C   0  0  2  0  0  0  0  0  0  0  0  0
    4.2392    0.2219    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    3.6897    0.4755    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    5.3370    0.4755    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    5.5918    0.2219    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    4.5133    0.9511    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.8660   -1.90

Now the structure stored in the "mysdf" can be used in a PUG-REST request through HTTP-POST.  For example, the code cell below shows how to retrieve various names (also called "synonyms") of the input structure.

In [35]:
url = prolog + "/compound/sdf/synonyms/txt"
mydata = { 'sdf': mysdf }
res = requests.post(url, data=mydata)
print(res.text)

morphine
Morphia
Morphium
Morphinum
Morphin
(-)-Morphine
Morphina
Ospalivina
DepoDur
57-27-2
Duromorph
Meconium
Morphinism
Moscontin
MS Contin
l-Morphine
Dulcontin
Morfina
Nepenthe
Roxanol
MORPHINE SULFATE
Infumorph
Dreamer
Morpho
Avinza
Hocus
Kadian
Unkie
Cube juice
Hard stuff
Statex SR
Ms Emma
Morphin [German]
Morfina [Italian]
Duramorph
Morphina [Italian]
M-Eslon
Morphine [BAN]
CCRIS 5762
HSDB 2134
Anhydrous morphine
(5R,6S,9R,13S,14R)-4,5-Epoxy-N-methyl-7-morphinen-3,6-diol
UNII-76I7G6D29C
CHEBI:17303
CHEMBL70
(5alpha,6alpha)-17-methyl-7,8-didehydro-4,5-epoxymorphinan-3,6-diol
4,5alpha-Epoxy-17-methyl-7-morphinen-3,6alpha-diol
7,8-Didehydro-4,5-epoxy-17-methyl-morphinan-3,6-diol
(7R,7AS,12BS)-3-METHYL-2,3,4,4A,7,7A-HEXAHYDRO-1H-4,12-METHANO[1]BENZOFURO[3,2-E]ISOQUINOLINE-7,9-DIOL
Morphine Anhydrate
76I7G6D29C
Morphine (BAN)
Dolcontin
RMS
(5alpha,6alpha)-Didehydro-4,5-epoxy-17-methylmorphinan-3,6-diol
Morphinan-3,6-alpha-diol, 7,8-didehydro-4,5-alpha-epoxy-17-methyl-
Morphinan-3,6-d

**Exercise 2a** Retrieve (in the **CSV** format) the XlogP, molecular weight, hydrogen bond donor count, hydrogen bond aceptor count, and TPSA of the compounds contained in the five sdf files below, which can be downloaded from the **Chapter 2 Assignments** page.  

- Use a for loop to retrieve the data for each compound.
- Import the time package and add "time.sleep(0.2)" to sleep 0.2 seconds after retrieving the data for each compound.
- Refer to the "lecture 1" notebook to see how to merge the multiple CSV outputs into a single CSV output.

In [36]:
files = ['lecture02_ex2b_compound1.sdf','lecture02_ex2b_compound2.sdf','lecture02_ex2b_compound3.sdf', \
         'lecture02_ex2b_compound4.sdf','lecture02_ex2b_compound5.sdf']

In [ ]:
# Write your code in this cell.


